In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Дисклеймер

Если кажется, что Вы выпали из повествования об основах классического машинного обучения, то эти два ноутбука проведут Вас за ручку от начала до текущего уровня, до которого мы успели дойти к концу 13-го занятия курса.

# Содержание

**А. Тренировочная задача** (первый ноутбук)<br>
1. Построим модель линейной регрессии на игрушечных данных "глазами";<br>
2. Посчитаем метрики качества модели;<br>
3. Построим модель линейной регрессии с помощью минимизации MSE;<br>
4. Посчитаем метрики качества и сравним с качеством первой модели, которую мы построили глазами.
<br><br>

**Б. Реальная задача** (**этот ноутбук**)<br>
1. Построим и оценим модель линейной регрессии для предсказания стоимости жилья только на числовых признаках;<br>
2. Добавим шкалирование числовых признаков и сравним метрики качества модели без шкалирования и с шкалированием;<br>
3. Добавим в лучшую из двух предыдущих моделей некоторые категориальные признаки;<br>
4. Добавим регуляризацию;<br>
5. Определим лучшие гиперпараметры с помощью K-fold кросс-валидации;<br>
6. Попробуем логарифмировать целевую переменную, заново обучим лучшую модель и сравним качество предсказания.<br>

# Б. Реальная задача

Будем работать с данными о стоимости жилья.<br>

Сами данные: `data/housing.csv`<br>
Аннотация (описания) колонок и их содержания: `data/data_description_housing.pdf`<br>

Посмотрите содержание аннотации.<br>
Как видите, там представлены числовые признаки, категориальные признаки, порядковые признаки (упорядоченные категории).<br>
Целевая переменная, `SalePrice`, находится в последнем столбце. Это фактическая стоиомость данного объекта недвижимости в у.е. Её мы и будем предсказывать.<br>
То есть мы по различными признакам пытаемся предсказать непрерывно распределенную величину, а значит -- это задача регрессии.

## 0. EDA

### 0.1 
* Прочитайте файл с данными. Разделитель -- запятая;
* Выведите первые 5 строк таблицы.

,Id,Type,Zone,LotArea,LotConfig,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,...,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PoolArea,MoSold,YrSold,SaleCondition,SalePrice
0,1820,70,RL,9986,Corner,1Fam,2Story,7,5,1915,...,Unf,3,661,TA,TA,0,2,2006,Abnorml,158511
1,33764,60,RL,8450,Inside,1Fam,2Story,7,5,1993,...,RFn,2,556,TA,TA,0,2,2007,Normal,253206
2,51420,50,RM,6060,Inside,1Fam,1.5Sto,7,5,1931,...,Unf,2,488,Fa,TA,0,4,2008,Abnorml,129501
3,147252,90,RL,10400,Inside,Duplex,1Story,4,5,1966,...,Unf,2,506,TA,TA,0,10,2006,Normal,78927
4,157900,20,RL,13975,Inside,1Fam,1Story,5,5,1998,...,Unf,2,565,TA,TA,0,6,2008,Normal,162064


### 0.2
* По файлу с аннотацией колонок определите числовые признаки;
* Создайте список с названиями числовых признаков и порядковых признаков, которые уже закодированы в числа (то есть которые уже являются числами в исходных данных).<br>
(можно оставить числовые колонки с помощью `df.select_dtypes(include='number')`);<br>
* В новую переменную отберите датафрейм только с числовыми колонками (**убедитесь, что вы не взяли таргет!**) ;<br>
Также уберите столбцы `Id` и `Type`;
* В отдельную переменную отберите таргет (`SalePrice`).

Размерность датафрейма только с числовыми колонками: (500, 21)


Размерность таргета: (500,)


## 1. Линейная модель на числовых признаках

### 1.1 
* Импортируйте из `sklearn.model_selection` функцию для разбиения данных на трейн и тест;<br>
* Разбейте датафрейм с числовыми признаками и таргет на трейн и тест;<br>
* Выделите в тест 20% данных;<br>
* Для воспроизводимости разбиения установите random_state=42.

X_train.shape: (400, 21)
y_train.shape: (400,)
X_test.shape: (100, 21)
y_test.shape: (100,)


### 1.2
* Импортируйте функцию модель линейной регрессии `LinearRegression`;
* Создайте экземпляр класса линейной регрессии;
* Обучите модель на выборке для обучения;
* Получите предсказания на тестовой выборке, сохраните их в отедльную переменную.

LinearRegression()

### 1.3
* Импортируйте из `sklearn.metric` функции для подсчёта MSE и $R^2$;
* Посчитайте импортированные метрики для предсказаний, полученных в пункте 1.2;
* Посчитайте RMSE (корень квадратный из MSE).

*_если у Вас получились идеальные метрики, значит столбец с таргетом попал в датафрейм с признаками_

MSE:  1482367295.9
RMSE: 38501.5
R^2:  0.689


## 2. Добавим шкалирование

### 2.1 
Для приведения признаков к одному масштабу мы воспользуемся стандартизацией. То есть из каждого числового столбца вычтем среднее признака и поделим на стандартное отклонение.<br>
Не будем делать это руками, а воспользуемся соответсвующим трансформером из `sklearn`.<br>

* Запустите ячейку ниже, чтобы все трансформеры возвращали не матрицу numpy, а датафрейм.<br>
Если этого не сделать, скейлер вернет вам "голую" матрицу без названия колонок.<br>
* Из `sklearn.preprocessing` импортируйте `StandardScaler`;
* Создайте экземпляр класса трансформера;
* Сделайте `fit` скейлера **только на данных для обучения**;
* Сделайте `transform` матрицы признаков для обучения и для теста, сохраните отшкалированные матрицы признаков в отдельные переменные (например *X_train_scaled*, *X_test_scaled*);
* Проверьте, что шкалирование прошло успешно. В исходных матрицах с признаками посчитайте средние значения для столбцов. Они будут разными.<br>
Если посчитать средние для шкалированных матрицах, то средние будут равны нулю или почти равны нулю.

In [10]:
from sklearn import set_config
set_config(transform_output = "pandas")

Средние для пяти столбцов из X_train


LotArea         10064.5600
OverallQual         6.0625
OverallCond         5.5825
YearBuilt        1967.8525
YearRemodAdd     1978.5600
dtype: float64

Средние для пяти столбцов из X_train_scaled


LotArea         1.015854e-16
OverallQual     2.220446e-17
OverallCond     3.708145e-16
YearBuilt       1.270095e-15
YearRemodAdd    2.613465e-15
dtype: float64

### 2.2 
* Обучите модель линейной регрессии на шкалированных признаках;
* Получите предсказания на тестовой выборке (тоже шкалированной);
* Посчитайте метрики качества (RMSE, $R^2$);
* Какая модель выдала лучший результат?

LinearRegression()

MSE:  1482367295.9
RMSE: 38501.5
R^2:  0.689


Не переживайте, если получились такие же метрики. **Так и должно быть**. Потому что предсказания линейной модели scale-invariant.<br>
Интуиция под этим простая: если мы поделили все значения признака в 2 раза, то коэффициент перед этим признаков просто увеличится в два раза. Та же логика и про центрирования.<br>
Таким образом, шкалирование признаков в линейных моделях не оказывает влияния на предсказания, так как эти операции компенсируются за счёт коэффициентов. <br>
Некоторые алгоритмы классичного машинного обучения, например метод к-ближайших соседей или метод опорных векторов, чувствительны к шкалированию и часто нуждаются в нём.<br><br>

**Зачем тогда шкалирования в линейных моделях?**<br>
1. Приведя признаки к одному масштабу, мы с помощью абсолютного значения коэффициентов можем оценить их вклад в предсказания. Таким образом, можем далее отобрать наиболее влиятельные признаки;<br>
2. Шкалирование признаков помогает градиентным методам лучше сходиться. Обычный LinearRegression объект найдет коэффициент и на нешкалированных признаках, а SGDRegressor будет лучше работать на шкалированных признаках.<br>

## 3. 
Давайте работать дальше на шкалированных признаках, чтобы мы могли оценить силу вклада признаков в предсказание с помощью коэффициентов.

### 3.1 
Сейчас у нас есть датафрейм с числовыми признаками, который мы разбивали на трейн и тест, а потом шкалировали. Сейчас мы из исходных данных отберем несколько категориальных столбцов, закодируем их, после этого отберем вместе с числовыми признаками в датафрейм, отшкалируем, обучим модель, оценим метрики. 

* Отберите их исходных данных в отдельную переменную категориальные колонки из списка `cat_cols` (он создан в ячейке ниже). Это категории с понятным порядком;
* С помощью метода `pd.Series.map` закодируйте каждый столбец в числа, сохранив порядок (пример в ячейке ниже для столбца `HouseStyle`;
* Сконкатенируйте закодированные категориальные столбцы с датафреймом с числовыми признаками (`pd.concat`, не забудьте указать нужную `axis`, вдоль которой конкатенировать).


In [17]:
cat_cols = ['HouseStyle', 'ExterQual', 'ExterCond', 'KitchenQual']

In [18]:
df_cat = df[cat_cols]
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   HouseStyle   500 non-null    object
 1   ExterQual    500 non-null    object
 2   ExterCond    500 non-null    object
 3   KitchenQual  500 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB


In [19]:
# в аннотации к данным можно посмотреть, какие категории есть в столбцах из cat_cols
df_cat['HouseStyle'].value_counts()

1Story    251
2Story    139
1.5Sto     65
2.5Sto     45
Name: HouseStyle, dtype: int64

In [20]:
# создаём словарь с соответствиями
# вместо каждого исходного значения в столбце
# запишем новое -- его числовое представление
map_dict = {
    '1Story':1, 
    '1.5Sto': 1.5, 
    '2Story':2,
    '2.5Sto':2.5
}

df_cat['HouseStyle'] = df_cat['HouseStyle'].map(map_dict)

<ipython-input-20-d6008bc5b0df>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['HouseStyle'] = df_cat['HouseStyle'].map(map_dict)


In [21]:
# видим, что теперь вместо категорий числа,
# олицетворяющие порядок категорий
df_cat.head(5)

,HouseStyle,ExterQual,ExterCond,KitchenQual
0,2.0,TA,TA,Gd
1,2.0,Gd,TA,Gd
2,1.5,TA,TA,TA
3,1.0,TA,TA,TA
4,1.0,TA,TA,Gd


In [22]:
# проделайте то же самое с оставшимися категориальными столбцами
# затем конкатенируйте закодированные категориальные столбцы к числовым

,HouseStyle,ExterQual,ExterCond,KitchenQual
0,2.0,0,0,1
1,2.0,1,0,1
2,1.5,0,0,0
3,1.0,0,0,0
4,1.0,0,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HouseStyle   500 non-null    float64
 1   ExterQual    500 non-null    int64  
 2   ExterCond    500 non-null    int64  
 3   KitchenQual  500 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 15.8 KB


(500, 21)

(500, 4)

(500, 25)

### 3.2
Давайте оценим, насколько изменится качество модели. 
* Разбейте датафрейм с числовыми и закодированными категориальными признаками на трейн и тест (используя тот же random_state и размер тестовой выборки, как раньше);
* Сделайте фит скейлера на трейне, трансфорт на трейне и тесте;
* Обучите модель линейной регрессии, получите предсказания, посчитайте метрики.

X_train.shape: (400, 25)
y_train.shape: (400,)
X_test.shape: (100, 25)
y_test.shape: (100,)


In [31]:
# у нас уже есть объект скейлера
# его можно не создавать заново, а просто переобучить


In [32]:
# также не нужно создавать заново модель
# она уже есть, просто переобучим её


MSE:  1317359878.9
RMSE: 36295.5
R^2:  0.724


Качество должно стать лучше: ошибка меньше, $R^2$ больше.<br>
_(*Держим в уме, что просто $R^2$ всегда становится тем больше, чем больше у нас признаков.<br>
Так что стоит считать adjusted $R^2$. Формулу можно найти в интернете. Она просто учитывает размер данных.)_

## 4 и 5.
Попробуем использовать регуляризацию и кросс-валидацию для выбора лучшей "силы" регуляризации.
* Импортируйте линейную модель с _l2_ регуляризацией<br>
(мы будем использовать _l2_ регуляризацию, так как в этом случае всё еще существует аналитическое решение, которому не нужно шкалирование. Шкалирование в `GridSearchCV` можно добавить только, создав `Pipeline`, но тогда код будет чуть сложнее.);
* Из подмодуля `model_selection` импортируйте `GridSearhCV` для поиска по сетке параметров;
* Создайте словарь с сеткой параметров (укажите 'cholesky' и 'svd' для параметра `solver` и 3-5 значений для силы регуляризации `alpha`);
* Создайте экземпляр `GridSearchCV`.<br>
При создании передайте ему модель с _l2_ регуляризацией, сетку параметров, установите установите 4 фолда для кросс-валидации.<br>
В качестве `scorer` сделайте `neg_mean_squared_error` (прямо в качетве аргумента дайте строку, `scorer="neg_mean_squared_error"`).
* Обучите созданный объект `GridSearchCV`. Потом посмотрите атрибуты `best_params_` и `best_score_`.<br>
В `best_score_` находится MSE, умноженная на минус 1. Так что, чтобы получить RMSE, надо сначала умноижть на -1, а потом взять корень квадратный.<br>
Можно также установить `verbose=2` или 3.

Fitting 4 folds for each of 16 candidates, totalling 64 fits
[CV 1/4] END alpha=0.01, solver=cholesky;, score=-1513421277.719 total time=   0.0s
[CV 2/4] END alpha=0.01, solver=cholesky;, score=-3283220736.301 total time=   0.0s
[CV 3/4] END alpha=0.01, solver=cholesky;, score=-1384450984.636 total time=   0.0s
[CV 4/4] END alpha=0.01, solver=cholesky;, score=-1720420876.390 total time=   0.0s
[CV 1/4] END ..alpha=0.01, solver=svd;, score=-1513421277.719 total time=   0.0s
[CV 2/4] END ..alpha=0.01, solver=svd;, score=-3283220736.301 total time=   0.0s
[CV 3/4] END ..alpha=0.01, solver=svd;, score=-1384450984.636 total time=   0.0s
[CV 4/4] END ..alpha=0.01, solver=svd;, score=-1720420876.390 total time=   0.0s
[CV 1/4] END alpha=0.1, solver=cholesky;, score=-1513475955.312 total time=   0.0s
[CV 2/4] END alpha=0.1, solver=cholesky;, score=-3283297610.442 total time=   0.0s
[CV 3/4] END alpha=0.1, solver=cholesky;, score=-1384070928.361 total time=   0.0s
[CV 4/4] END alpha=0.1, solver

GridSearchCV(cv=4, estimator=Ridge(),
             param_grid={'alpha': [0.01, 0.1, 0.5, 1, 2.5, 10, 100, 1000],
                         'solver': ['cholesky', 'svd']},
             scoring='neg_mean_squared_error', verbose=3)

{'alpha': 100, 'solver': 'svd'}

43946.7248835305

## 6.
Иногда шкалирование целевой переменной может улучшить качество модели. Например, если цена квартиры растет нелинейно, а экспоненциально, то логарифмировав целевую переменную, мы получим степени. А степени в таком случае меняются линейно.<br>

Более развернутое пояснение. Представим, что каждый квадратный метр удваивает стоимость квартиры. Тогда прибавив 5 квадартных метров, мы сделаем квартиру дороже в 2**5 раз.<br>
То есть если увеличить число квадратных метров НА 1, цена растет ВО сколько-то раз. Надо, чтобы при увеличении метров НА сколько-то единиц, целевая переменная тоже росла НА сколько-то единиц.<br>
Если мы логарифмируем цены, то тогда при добавлении 1 квадратного метра, логарифм цена растет НА 1.<br> 
Логарифм превращает умножение в сумму. 

### 6.1
Давайте для простоты вернемся к датафрейму с числовыми и категориальными признаками. Он уже разбит на трейн и тест и отшкалирован.<br>
* Логарифмируйте целевую переменную для трейна (воспользуйте просто `np.log`, основание этого логарифма -- экспонента);
* Обучите обычную линейную модель без регуляризации на откашлированных данных для трейна из пункта 3.2;
* Получите предскаазния для теста.<br>
Так как мы учились на на ценах квартир, а на степенях, в которвую надо возвести экспоненту, чтобы получить цену квартиры, то и предсказывает наша модель степени.
* Так как мы получили степени экспоненты, то надо возвести экспоненту в соответствующие степени, чтобы получить цены квартир (дайте функции `np.exp` ваши предсказания, чтобы из степеней получить цены);
* Посчитайте метрики, сравните с метриками из пункта 3.2.

In [38]:
# также не нужно создавать заново модель
# она уже есть, просто переобучим её


MSE:  1273486599.3
RMSE: 35685.9
R^2:  0.733


Метрики должны были стать чуть лучше.

# Take home messages